# Inspect best model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *

from matplotlib.colors import LinearSegmentedColormap

from captum.attr import Occlusion
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import visualization as viz

In [3]:
CONFIG = {
    "dir_dataset": Path("./clean_dataset"),
    "dir_images": Path("./clean_dataset/images"),
    "trained_weights": "resnet34_unfreeze_img_size_252_DDD-36",
    "validation_set": "valid.txt",
    "model": models.resnet34,
    "metric": [error_rate, accuracy],
    "img_size": 252,
    "batch_size": 64,
}

In [4]:
# function for getting label information out of train file name
# Example: p002_c0_img_3370 -> c0
get_y_fn = lambda x: x.stem.split("_")[1]

In [5]:
# create Databunch
data = (ImageList.from_folder(CONFIG["dir_dataset"])
        .split_by_fname_file(CONFIG["validation_set"])
        .label_from_func(get_y_fn)
        .transform(get_transforms(do_flip=False), size=CONFIG["img_size"])
        .databunch(bs=CONFIG["batch_size"])
        .normalize(imagenet_stats)
    )

In [6]:
learn = cnn_learner(data, CONFIG["model"], metrics=CONFIG["metric"])

In [7]:
learn.load(CONFIG["trained_weights"]) ;